# Imports

In [ ]:
import polars as pl
from transformers import AutoTokenizer, AutoModel

# Load data

In [2]:
df = pl.read_parquet("home/ubuntu/sample")

# Load model

In [3]:
model_name = "huawei-noah/TinyBERT_General_4L_312D"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to('cuda')

C:\Users\Минтимер\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# Embed concepts

In [5]:
import numpy as np
for part in range(0, len(df['high_concepts_flat']), 200_000):
    embs = []
    for i in range(part, min(part + 200_000, len(df['high_concepts_flat'])), 500):
        print(i)
        sentences = list(df['high_concepts_flat'][i:i + 500])
    
        inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True, max_length=128).to('cuda')
        outputs = model(**inputs)
        
        cls_embeddings = outputs.last_hidden_state[:, 0, :]
        embs.append(cls_embeddings.cpu().detach().numpy())
    embs = np.concatenate(embs)
    np.save(str(part),embs)

1000000
1000500
1001000
1001500
1002000
1002500
1003000
1003500
1004000
1004500
1005000
1005500
1006000
1006500
1007000
1007500
1008000
1008500
1009000
1009500
1010000
1010500
1011000
1011500
1012000
1012500
1013000
1013500
1014000
1014500
1015000
1015500
1016000
1016500
1017000
1017500
1018000
1018500
1019000
1019500
1020000
1020500
1021000
1021500
1022000
1022500
1023000
1023500
1024000
1024500
1025000
1025500
1026000
1026500
1027000
1027500
1028000
1028500
1029000
1029500
1030000
1030500
1031000
1031500


# Save concept embeddings

In [7]:
from os import listdir

data = np.concatenate([np.load(f'parts/{part}') for part in listdir('parts')])
np.save('concepts', data)